In [21]:
import struct
import array
import zlib
import numpy as np
import xarray as xr


In [7]:
def read_gdb_file(file_path):
    with open(file_path, 'rb') as file:

        header = file.read(24)
        #file_code, = struct.unpack('<i', header[:4])
        file_description = header[4:24].decode('utf-8').strip('\x00')
        
        magic_number = struct.unpack("<I", file.read(4))[0]
        
        #print(f"File Code: {file_code}")
        print(f"File Description: {file_description}")
        
        print(f"Magic Number: {magic_number}")

# Use the function
read_gdb_file('data/DB_1116.gdb')

File Description: 
Magic Number: 50


In [32]:
def read_geosoft_gdb(file_path):
    with open(file_path, 'rb') as file:
        # Read the header (example: assuming the header is 256 bytes)
        header = file.read(256)
        print(header[10:13])
        print(header)
        grid_sub = zlib.decompress(header[10:13])
        print(grd_sub)
        
        # Unpack the header (example: assuming the header contains an integer and a float)
        #header_data = struct.unpack('if', header[:8])
        header_data = struct.unpack('ssss', header[:4])
        print("Header Data:", header_data)
        
        # Read the rest of the file (example: assuming the rest of the file contains records of 16 bytes each)
        while True:
            record = file.read(16)
            break
            if not record:
                break
            # Unpack the record (example: assuming each record contains two integers and two floats)
            record_data = struct.unpack('iiff', record)
            print("Record Data:", record_data)

# Example usage
read_geosoft_gdb('data/DB_1116.gdb') 

b'\x02\x10\x08'
b'!CBD\x00\x00\x00\x00\x00\x00\x02\x10\x08\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x002\x00\x00\x00^\x01\x00\x00d\x00\x00\x00\xc8\x00\x00\x00\n\x00\x00\x00\xdc(\x00\x00\x10\'\x00\x00n(\x00\x00x(\x00\x00x(\x00\x00b\x02\x00\x00\x00\x00\x00\x00^\x01\x00\x00&\x02\x00\x00X\x02\x00\x00^\x01\x00\x00\xc8\x00\x00\x002\x00\x00\x00\n\x00\x00\x00\x00\x04\x00\x00@\'\x02\x00\x8a\x00\x00\x00"=\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


error: Error -3 while decompressing data: incorrect header check

In [26]:
header.shape

NameError: name 'header' is not defined

In [17]:
62416 - 21968

40448

In [19]:
40448 / 256

158.0

NameError: name 'header' is not defined